# Libraries and preparation

refs:
- https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_rag_agent_llama3_local.ipynb
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb?ref=blog.langchain.dev
- https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb?ref=blog.langchain.dev

Starting ollama server and pulling llama3

MAP:REDUCE: https://langchain-ai.github.io/langgraph/how-tos/map-reduce/

In [ ]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
t.start()

In [ ]:
!ollama pull llama3

In [ ]:
t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", "llama3"]),daemon=True)
t2.start()

In [ ]:
%%capture --no-stderr
%pip install -U langchain-ai21 langchain-pinecone langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic[local] langchain-text-splitters

In [ ]:
# choosing llm

local_llm = "llama3"

In [1]:
# Tracing and api-keys
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["TAVILY_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "94ef7896-1fae-44d3-b8d2-0bd6f5f664f5"
os.environ["AI21_API_KEY"] = "KlINkh5QKw3hG1b5Hr75YDO7TwGoQvzn"

# Tools

refs:
- https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/

In [ ]:
### Search
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

# Indexing

Organizing external sources for the llm. Phase of indexing and chunking of docs refs:
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/
- https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/
- https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
- Nomic embeddings: https://docs.nomic.ai/atlas/capabilities/embeddings#selecting-a-device

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_ai21 import AI21Embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls] #text + meta-data on docs
docs_list = [item for sublist in docs for item in sublist] #ci serve l'attributo page_content

# Chunking
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)
index_name = "langchain-test-index"

# Add to vectorDB
vectorstore = PineconeVectorStore.from_documents(
    documents=doc_splits,
    #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local", device="cuda"),
    embedding=AI21Embeddings(device="cuda"),
    index_name=index_name
)
retriever = vectorstore.as_retriever()

# Indexing KBT

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_ai21 import AI21Embeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [#TODO
]

docs = [WebBaseLoader(url).load() for url in urls] #text + meta-data on docs
docs_list = [item for sublist in docs for item in sublist] #ci serve l'attributo page_content

# Chunking
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)
index_name = "langchain-test-index"

# Add to vectorDB
vectorstore_KBT = PineconeVectorStore.from_documents(
    documents=doc_splits,
    #embedding=NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local", device="cuda"),
    embedding=AI21Embeddings(device="cuda"),
    index_name=index_name
)
retriever_KBT = vectorstore_KBT.as_retriever()

# Query generation (multi-aspects)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant that generates multiple
    search queries based on some specified aspects from a single input query. You have to generate ONLY ONE query
    for EACH aspect. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Generate search aspect-queries related to the original query: {original_query}.
    Where the aspects are: {aspects}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["aspects","original_query"],
)

llm = ChatOllama(model=local_llm, format="json", temperature=0)

query_generator = prompt | llm | JsonOutputParser()

# Prova

# original_query = "What about Covid19?"
# generation = query_generator.invoke({"original_query": original_query})
# print(generation)


# # Reciprocal Rank Fusion algorithm
# def reciprocal_rank_fusion(search_results_dict, k=60):
#     fused_scores = {}
#     print("Initial individual search result ranks:")
#     for query, doc_scores in search_results_dict.items():
#         print(f"For query '{query}': {doc_scores}")

#     for query, doc_scores in search_results_dict.items():
#         for rank, (doc, score) in enumerate(sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)):
#             if doc not in fused_scores:
#                 fused_scores[doc] = 0
#             previous_score = fused_scores[doc]
#             fused_scores[doc] += 1 / (rank + k)
#             print(f"Updating score for {doc} from {previous_score} to {fused_scores[doc]} based on rank {rank} in query '{query}'")

#     reranked_results = {doc: score for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)}
#     print("Final reranked results:", reranked_results)
#     return reranked_results

# Organizing outputs

In [ ]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant that organizes and puts together the answers from multiple
    queries based on different aspects in a single answer. Make the concatenations or a summary. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the answers: {answers}
    Result answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["answers"],
)

llm = ChatOllama(model=local_llm, format="json", temperature=0)

final_answer = prompt | llm | JsonOutputParser()
#final_output = final_answer.invoke({"answers": answers})
#print(final_output)

answers= ["""You didn't ask a specific question, but I can summarize the text for you.
 \n\nThe article discusses the economic impact of COVID-19 around the world. 
 It highlights that the pandemic has had significant and persistent effects on output and prices,
 especially in emerging and developing countries. The article also mentions that the partnership
 with BioNTech, high cost of production and management of the vaccine, and growing number of 
 same-size competitors have reduced investors' trust in the company to have bigger revenue 
 in 2021.\n\nThe article further notes that global coordination and cooperation are crucial for 
 slowing the spread of the pandemic and alleviating economic damage. It also emphasizes the 
 humanitarian and economic toll the global recession will take on economies with extensive 
 informal sectors, which make up an estimated one-third of GDP and about 70% of total employment 
 in emerging market and developing economies.\n\nThe article also discusses the long-term damage 
 to potential output and productivity growth, citing the June 2020 Global Economic Prospects 
 report. The report assumes that the pandemic recedes in such a way that domestic mitigation 
 measures can be lifted by mid-year in advanced economies and later in developing countries, 
 and that adverse global spillovers ease during the second half of 2020.\n\nFinally, the article 
 mentions the UN's latest World Economic Situation and Prospects report, which projects the world 
 economy to grow by 2.3% in 2023 and 2.5% in 2024, with the least developed countries forecast 
 to grow by 4.1% in 2023 and 5.2% in 2024.\n\nIf you have a specific question or would like me to 
 answer something related to this text, feel free to ask!", "It seems like you didn't provide a 
 specific question for me to answer. The text appears to be a scientific article and a brief 
 report on the impact of COVID-19 pandemic on social interactions and economic well-being.
 \n\nIf you have a specific question related to this topic, I'd be happy to help."""]

# Retrieval

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model=local_llm, format="json", temperature=0) #higher temperature more likely hallucinations

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
# question = "agent memory"
# docs = retriever.invoke(question)
# doc_txt = docs[1].page_content
# print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

# Generating answer

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)


# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
# question = "agent memory"
# docs = retriever.invoke(question)
# generation = rag_chain.invoke({"context": docs, "question": question})
# print(generation)

# Hallucinations check

Per ora non uso hallucinations check

In [ ]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' (both in lower case) to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    SINGLE KEY 'score' and NO preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
#hallucination_grader.invoke({"documents": docs, "generation": generation})

# Answer check

In [ ]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' (both in lower case) to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a SINGLE KEY 'score' and NO preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
#answer_grader.invoke({"question": question, "generation": generation})

# Routing

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
    user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents,
    prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and
    no premable or explanation. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()
# question = "llm agent memory"
# docs = retriever.get_relevant_documents(question)
# doc_txt = docs[1].page_content
# print(question_router.invoke({"question": question}))

# Document check (Entailment)

In [ ]:
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You have to perform a task of textual entailments
    between two documents. Give a binary score 'yes' or 'no' (both in lower case) to indicate whether the second document is
    entailed by the first document. Provide the binary score as a JSON with a SINGLE KEY 'score' and NO preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> The first document:
    {first_doc}
    \n ------- \n
    Entails with:
    \n ------- \n
    {second_doc}
    \n ------- \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["first_doc", "second_doc"],
)

entailment_checker = prompt | llm | JsonOutputParser()

# 

# Control flow (Aspect agents)

refs
- https://www.langchain.com/langgraph

In [ ]:
from pprint import pprint
from typing import List, Annotated
import operator

from langchain_core.documents import Document
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph, START

### State
class GraphState(TypedDict):
    """
    Represents the state of graph of aspect agents.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
    
    query: str
    answers_agent: Annotated[List[str], operator.add]
    my_answer: str
    web_search: str
    documents: List[str]
    documents_kbt: List[str]


def retrieve(state):
    """
    Retrieve documents from vectorstore and from KBT

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    print(f"State: {state}")
    queries = state["query"]

    # Retrieval
    documents = retriever.invoke(query)
    documents_kbt = retriever_KBT.invoke(query)
    return {"documents": documents, "documents_kbt": documents_kbt, "query": query}


def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": query})
    return {"documents": documents, "query": query, "my_answer": generation, "answers_agent": [generation]}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": query, "document": d.page_content}
        )
        grade = score["score"]
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "query": query, "web_search": web_search}


def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": query})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "query": query}

def entailment_filter(state):
    """
    Filter documents that doesn't entail with KBT

    Args:
        state (dict): The current graph state
    """

    print("---ENTAILMENT FILTER---")
    print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    documents_KBT = state["documents_KBT"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        for d_kbt in documents:
            score = entailment_checker.invoke(
                {"first_doc": d_kbt.page_content, "second_doc": d.page_content}
            )
            grade = score["score"]
            # Document entailed
            if grade.lower() == "yes":
                print("---DOCUMENT ENTAILED---")
                filtered_docs.append(d)
                break
    return {"documents": filtered_docs}


### Conditional edge

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    print(f"State: {state}")
    query = state["query"]
    #print(queries)
    source = question_router.invoke({"question": query})
    #print(source)
    #print(source["datasource"])
    if source["datasource"] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    print(f"State: {state}")
    web_search = state["web_search"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"



def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    print(f"State: {state}")
    query = state["query"]
    documents = state["documents"]
    my_answer = state["my_answer"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": my_answer} #answers_agent[0]
    )
    print(f"score: {score}")
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": query, "generation": my_answer})
        grade = score["score"]
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

**Building graph with edges**

In [ ]:
# Build graph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("entailment_filter", entailment_filter)  # generatae

workflow.add_conditional_edges(
    START,
    route_question,
    {
        "websearch": "websearch", #se la risposta è websearch, allora vai al nodo websearch
        "vectorstore": "retrieve", #se la risposta è vectorstore, allora vai al nodo retrieve
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "entailment_filter",
    },
)
workflow.add_edge("websearch", "entailment_filter")

workflow.add_edge("entailment_filter", "generate")

# Per ora non faccio il controllo sulle allucinazioni
workflow.add_edge("generate", END)
"""
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)
"""
    
workflow_compiled = workflow.compile()

In [ ]:
#for i in range(num_aspects):
#    print(workflows[i].config_specs)

In [ ]:
from IPython.display import Image, display

display(Image(workflow_compiled.get_graph().draw_mermaid_png()))

In [ ]:
# Test

inputs = {"query": "What about COVID19?"}
#print(workflows[0].stream(inputs))
for output in workflow_compiled.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["my_answer"])
#workflows[0].invoke({"queries": ["What about COVID19?"]},{"id_agent":1})

# Application

In [ ]:
from typing import Annotated
import operator
from langgraph.constants import Send


### Super Graph State
class SuperGraphState(TypedDict):
    """
    Represents the state of our super-graph.

    Attributes:
        question: question
        generation: LLM generation
        agents_answer: generation of agents
        answer: the final answer
    """
    
    question: str
    queries: List[str]
    answers_agent: Annotated[List[str], operator.add]
    final_answer: str


def generate_queries(state):
    """
    Generate multi-aspect queries from the starting question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains the multi aspect queries
    """
    print("---GENERATE MULTI-ASPECTS QUERIES---")
    print(f"State: {state}")
    question = state["question"]
    aspects = ["Health","Economy","Society"]

    generation = query_generator.invoke({"aspects": aspects, "original_query": question})
    #print(list(generation.values()))
    return {"queries": list(generation.values())}

def send_queries(state):
    return [Send("aspect_agent_node", {"query": q}) for q in state["queries"]]

def organize_outputs(state):
    """
    Organize the outputs of the agents.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, answer, that contains the final answer to give to user
    """
    print("---ORGANIZE OUTPUTS---")
    print(f"State: {state}")
    answers_agent = state["answers_agent"]

    final_output =final_answer.invoke({"answers": answers_agent})
    return {"final_answer": final_output}

master_flow = StateGraph(SuperGraphState)

# Define the nodes
master_flow.add_node("generate_queries", generate_queries)
master_flow.add_node("organize_queries", organize_outputs)
master_flow.add_node("aspect_agent_node",workflow_compiled)

# Build graph
master_flow.add_edge(START, "generate_queries")
master_flow.add_conditional_edges("generate_queries", send_queries, ["aspect_agent_node"])
master_flow.add_edge("aspect_agent_node", "organize_queries")
master_flow.add_edge("organize_queries", END)

app = master_flow.compile()

In [ ]:
from IPython.display import Image, display

# Setting xray to 1 will show the internal structure of the nested graph
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# Test

inputs = {"question": "What about COVID19?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["final_answer"])